<a href="https://colab.research.google.com/github/Aradhya6394/Artificial-Intelligence-2025/blob/main/Min_Max_Alphabeta_Huristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Tic-Tac-Toe with depth-limited Minimax and a heuristic "expected utility" evaluator.

- 'X' is the maximizing player.
- 'O' is the minimizing player.
- board is a list of 9 elements: 'X', 'O', or ' ' (space) for empty.
- get_best_move(board, player, max_depth) returns the best move index (0..8)
  for 'player' computed by Minimax with depth cutoff and heuristic estimate.

Heuristic used for cutoff:
- +100 for X win, -100 for O win, 0 for draw (terminal).
- For non-terminal nodes at cutoff, evaluate each line (row/col/diag):
    * If line contains only X and empty: add + (10^(num_X)) weights:
        - 1 X => +1
        - 2 X => +10
    * If line contains only O and empty: subtract similarly.
This yields an estimate of "expected utility" indicating how favourable the board is.
"""
from typing import List, Tuple, Optional

LINES = [
    (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rows
    (0, 3, 6), (1, 4, 7), (2, 5, 8),  # cols
    (0, 4, 8), (2, 4, 6)              # diags
]


def is_terminal(board: List[str]) -> Tuple[bool, Optional[str]]:
    """Return (True, 'X'/'O'/'Draw') if terminal, else (False, None)."""
    for a, b, c in LINES:
        if board[a] != ' ' and board[a] == board[b] == board[c]:
            return True, board[a]
    if ' ' not in board:
        return True, 'Draw'
    return False, None


def heuristic_expected_utility(board: List[str]) -> int:
    """
    Heuristic estimate of utility for non-terminal states.
    Positive => good for X (maximizer). Negative => good for O (minimizer).
    """
    score = 0
    for a, b, c in LINES:
        line = [board[a], board[b], board[c]]
        if line.count('X') > 0 and line.count('O') > 0:
            # contested line -> no immediate potential
            continue
        # only X and empties
        if line.count('O') == 0:
            nx = line.count('X')
            if nx == 1:
                score += 1        # small potential
            elif nx == 2:
                score += 10       # strong potential
        # only O and empties
        if line.count('X') == 0:
            no = line.count('O')
            if no == 1:
                score -= 1
            elif no == 2:
                score -= 10
    return score


def utility_value(winner: str) -> int:
    """Terminal utility values."""
    if winner == 'X':
        return 100
    if winner == 'O':
        return -100
    return 0  # draw


def available_moves(board: List[str]) -> List[int]:
    return [i for i, v in enumerate(board) if v == ' ']


def minimax(board: List[str],
            depth: int,
            max_depth: int,
            maximizing: bool,
            alpha: int = -10**9,
            beta: int = 10**9) -> Tuple[int, Optional[int]]:
    """
    Depth-limited Minimax with alpha-beta pruning.
    Returns (best_score, best_move_index).
    If best_move_index is None => no moves (terminal or cutoff without choice).
    """
    term, winner = is_terminal(board)
    if term:
        return utility_value(winner), None

    if depth >= max_depth:
        # Cutoff: return heuristic estimate (expected utility)
        return heuristic_expected_utility(board), None

    moves = available_moves(board)
    best_move = None

    if maximizing:
        best_score = -10**9
        for m in moves:
            board[m] = 'X'
            score, _ = minimax(board, depth + 1, max_depth, False, alpha, beta)
            board[m] = ' '
            if score > best_score:
                best_score = score
                best_move = m
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break  # beta cut-off
        return best_score, best_move
    else:
        best_score = 10**9
        for m in moves:
            board[m] = 'O'
            score, _ = minimax(board, depth + 1, max_depth, True, alpha, beta)
            board[m] = ' '
            if score < best_score:
                best_score = score
                best_move = m
            beta = min(beta, best_score)
            if beta <= alpha:
                break  # alpha cut-off
        return best_score, best_move


def get_best_move(board: List[str], player: str, max_depth: int = 4) -> Optional[int]:
    """
    Public helper to obtain best move for 'player' ('X' or 'O') with depth cutoff.
    max_depth: maximum search depth (0 means only heuristic evaluation).
    """
    maximizing = (player == 'X')
    score, move = minimax(board, depth=0, max_depth=max_depth, maximizing=maximizing)
    # Note: when cutoff occurs at root (max_depth==0), move can be None -> choose best heuristically
    if move is None:
        # fallback: choose move that maximizes heuristic locally
        best_move = None
        best_score = -10**9 if maximizing else 10**9
        for m in available_moves(board):
            board[m] = player
            s = heuristic_expected_utility(board)
            board[m] = ' '
            if maximizing and s > best_score:
                best_score, best_move = s, m
            if not maximizing and s < best_score:
                best_score, best_move = s, m
        return best_move
    return move


# ---------- Utilities for demonstration ----------
def pretty_print(board: List[str]):
    for r in range(3):
        print(' | '.join(board[3 * r:3 * r + 3]))
        if r < 2:
            print('---------')


if __name__ == "__main__":
    # Example: partially-filled board
    # Indexes:
    # 0 1 2
    # 3 4 5
    # 6 7 8
    board = ['X', 'O', 'X',
             ' ', 'O', ' ',
             ' ', ' ', ' ']

    print("Current board:")
    pretty_print(board)
    print()

    # Get best move for X with depth limit 4
    best_move = get_best_move(board, 'X', max_depth=4)
    print(f"Best move for X (depth limit 4): {best_move}")
    if best_move is not None:
        board[best_move] = 'X'
        print("Board after recommended move:")
        pretty_print(board)


Current board:
X | O | X
---------
  | O |  
---------
  |   |  

Best move for X (depth limit 4): 7
Board after recommended move:
X | O | X
---------
  | O |  
---------
  | X |  


In [4]:
from typing import List, Tuple, Optional

# ---------------- GAME LOGIC ----------------

# All possible winning combinations (rows, columns, diagonals)
LINES = [
    (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rows
    (0, 3, 6), (1, 4, 7), (2, 5, 8),  # columns
    (0, 4, 8), (2, 4, 6)              # diagonals
]


def is_terminal(board: List[str]) -> Tuple[bool, Optional[str]]:
    """
    Check if the game has ended.
    Returns (True, winner) if terminal, else (False, None).
    Winner can be 'X', 'O', or 'Draw'.
    """
    # Check for a winner
    for a, b, c in LINES:
        if board[a] != ' ' and board[a] == board[b] == board[c]:
            return True, board[a]

    # If no empty spaces left and no winner, it's a draw
    if ' ' not in board:
        return True, 'Draw'

    # Game still ongoing
    return False, None


def heuristic_expected_utility(board: List[str]) -> int:
    """
    Heuristic evaluation function.
    Estimates how good the current board is for player X.
    (Higher = better for X, lower = better for O)
    """
    score = 0
    for a, b, c in LINES:
        line = [board[a], board[b], board[c]]

        # If both players occupy the same line, ignore it (no advantage)
        if line.count('X') > 0 and line.count('O') > 0:
            continue

        # If only Xs are in the line → give positive score
        if line.count('O') == 0:
            nx = line.count('X')
            if nx == 1:
                score += 1     # One X → small advantage
            elif nx == 2:
                score += 10    # Two Xs → strong advantage

        # If only Os are in the line → give negative score
        if line.count('X') == 0:
            no = line.count('O')
            if no == 1:
                score -= 1     # One O → small disadvantage
            elif no == 2:
                score -= 10    # Two Os → big disadvantage

    return score


def utility_value(winner: str) -> int:
    """
    Returns the final utility value for a terminal (end) state.
    +100 for X win, -100 for O win, 0 for Draw.
    """
    if winner == 'X':
        return 100
    if winner == 'O':
        return -100
    return 0


def available_moves(board: List[str]) -> List[int]:
    """Return list of empty positions where a move can be made."""
    return [i for i, v in enumerate(board) if v == ' ']


def minimax(board: List[str], depth: int, max_depth: int, maximizing: bool,
            alpha: int = -10**9, beta: int = 10**9) -> Tuple[int, Optional[int]]:
    """
    Minimax algorithm with Alpha-Beta pruning.
    Used by the AI to find the optimal move.
    """
    # Check if game has ended
    term, winner = is_terminal(board)
    if term:
        return utility_value(winner), None

    # If max depth reached → evaluate board using heuristic
    if depth >= max_depth:
        return heuristic_expected_utility(board), None

    moves = available_moves(board)
    best_move = None

    # ---------------- MAXIMIZING PLAYER (X) ----------------
    if maximizing:
        best_score = -10**9
        for m in moves:
            board[m] = 'X'  # Make the move
            score, _ = minimax(board, depth + 1, max_depth, False, alpha, beta)
            board[m] = ' '  # Undo the move

            # Choose move with highest score
            if score > best_score:
                best_score = score
                best_move = m

            # Alpha-beta pruning
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break

        return best_score, best_move

    # ---------------- MINIMIZING PLAYER (O) ----------------
    else:
        best_score = 10**9
        for m in moves:
            board[m] = 'O'  # Make the move
            score, _ = minimax(board, depth + 1, max_depth, True, alpha, beta)
            board[m] = ' '  # Undo the move

            # Choose move with lowest score
            if score < best_score:
                best_score = score
                best_move = m

            # Alpha-beta pruning
            beta = min(beta, best_score)
            if beta <= alpha:
                break

        return best_score, best_move


def get_best_move(board: List[str], player: str, max_depth: int = 4) -> Optional[int]:
    """
    Finds the best move for the given player ('X' or 'O').
    Uses minimax search (with fallback to heuristic if needed).
    """
    maximizing = (player == 'X')
    score, move = minimax(board, 0, max_depth, maximizing)

    # If minimax couldn’t find a move (rare case), fallback to heuristic evaluation
    if move is None:
        best_move = None
        best_score = -10**9 if maximizing else 10**9

        for m in available_moves(board):
            board[m] = player
            s = heuristic_expected_utility(board)
            board[m] = ' '
            if maximizing and s > best_score:
                best_score, best_move = s, m
            elif not maximizing and s < best_score:
                best_score, best_move = s, m
        return best_move

    return move


# ---------------- DISPLAY FUNCTIONS ----------------

def pretty_print(board: List[str]):
    """
    Display the Tic-Tac-Toe board in a readable grid format.
    """
    print("\nBoard positions:")
    print("0 | 1 | 2")
    print("---------")
    print("3 | 4 | 5")
    print("---------")
    print("6 | 7 | 8\n")

    print("Current board:")
    for r in range(3):
        print(' | '.join(board[3 * r:3 * r + 3]))
        if r < 2:
            print("---------")


# ---------------- MAIN GAME LOOP ----------------

def play_game():
    """
    Main function for Human vs Computer Tic-Tac-Toe.
    Handles user input, AI decisions, and displays the game progress.
    """
    print("Welcome to Tic-Tac-Toe with AI!")
    board = [' '] * 9  # Empty 3x3 board

    # Let user choose their symbol
    human = input("Choose your symbol (X or O): ").upper()
    while human not in ['X', 'O']:
        human = input("Please choose either X or O: ").upper()

    ai = 'O' if human == 'X' else 'X'
    current_player = 'X'  # X always starts first

    print(f"\nYou are {human}. Computer is {ai}. Let's play!\n")

    while True:
        # Display board
        pretty_print(board)

        # Check if game is over
        term, winner = is_terminal(board)
        if term:
            print("\nGame Over!")
            if winner == 'Draw':
                print("It's a Draw 😐")
            elif winner == human:
                print("🎉 You Win! 🎉")
            else:
                print("💻 Computer Wins! 💻")
            break

        # ---------------- HUMAN TURN ----------------
        if current_player == human:
            try:
                move = int(input("Enter your move (0-8): "))
            except ValueError:
                print("Invalid input. Please enter a number between 0 and 8.")
                continue

            # Validate move
            if move not in available_moves(board):
                print("That cell is already taken or invalid! Try again.")
                continue

            board[move] = human

        # ---------------- AI TURN ----------------
        else:
            print("\nComputer is thinking...")
            move = get_best_move(board, ai, max_depth=4)
            board[move] = ai
            print(f"Computer chose position {move}")

        # Switch turn between players
        current_player = 'O' if current_player == 'X' else 'X'


# ---------------- RUN GAME ----------------

if __name__ == "__main__":
    play_game()


Welcome to Tic-Tac-Toe with AI!
Choose your symbol (X or O): x

You are X. Computer is O. Let's play!


Board positions:
0 | 1 | 2
---------
3 | 4 | 5
---------
6 | 7 | 8

Current board:
  |   |  
---------
  |   |  
---------
  |   |  
Enter your move (0-8): 4

Board positions:
0 | 1 | 2
---------
3 | 4 | 5
---------
6 | 7 | 8

Current board:
  |   |  
---------
  | X |  
---------
  |   |  

Computer is thinking...
Computer chose position 0

Board positions:
0 | 1 | 2
---------
3 | 4 | 5
---------
6 | 7 | 8

Current board:
O |   |  
---------
  | X |  
---------
  |   |  
Enter your move (0-8): 6

Board positions:
0 | 1 | 2
---------
3 | 4 | 5
---------
6 | 7 | 8

Current board:
O |   |  
---------
  | X |  
---------
X |   |  

Computer is thinking...
Computer chose position 2

Board positions:
0 | 1 | 2
---------
3 | 4 | 5
---------
6 | 7 | 8

Current board:
O |   | O
---------
  | X |  
---------
X |   |  
Enter your move (0-8): 1

Board positions:
0 | 1 | 2
---------
3 | 4 | 5
--